# Notebook 110: 最適化の基礎 ― 勾配降下法を理解する

## Optimization Fundamentals: Understanding Gradient Descent

---

### このノートブックの位置づけ

**Phase 10「最適化手法」** の第1章として、機械学習における最適化の基礎と勾配降下法の本質を理解します。

### 学習目標

1. **最適化問題** の定式化を理解する
2. **勾配降下法** のアルゴリズムと直感を習得する
3. **学習率** の役割と影響を理解する
4. **バッチサイズ** と確率的勾配降下法の関係を理解する
5. **損失曲面** の可視化と最適化の挙動を観察する

### 前提知識

- 微分の基礎（Notebook 70）
- 勾配ベクトルの概念

---

## 目次

1. [最適化問題とは](#1-最適化問題とは)
2. [勾配降下法の原理](#2-勾配降下法の原理)
3. [学習率の影響](#3-学習率の影響)
4. [バッチ勾配降下法 vs 確率的勾配降下法](#4-バッチ勾配降下法-vs-確率的勾配降下法)
5. [損失曲面の可視化](#5-損失曲面の可視化)
6. [凸関数と非凸関数](#6-凸関数と非凸関数)
7. [収束条件と停止基準](#7-収束条件と停止基準)
8. [演習問題](#8-演習問題)
9. [まとめと次のステップ](#9-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 最適化問題とは

### 1.1 機械学習における最適化

機械学習の本質は **最適化問題** を解くことです。

$$
\theta^* = \arg\min_{\theta} L(\theta)
$$

- $\theta$: モデルのパラメータ（重み、バイアス）
- $L(\theta)$: 損失関数（目的関数）
- $\theta^*$: 最適なパラメータ

### 1.2 最適化の課題

1. **高次元空間**: ニューラルネットワークは数百万〜数十億のパラメータを持つ
2. **非凸性**: 損失曲面には多くの局所解が存在
3. **計算コスト**: 大規模データセットでの効率的な計算

In [ ]:
# 最適化問題の例: 2次関数の最小化

def quadratic(x):
    """f(x) = x² - 4x + 4 = (x - 2)²"""
    return x**2 - 4*x + 4

def quadratic_grad(x):
    """f'(x) = 2x - 4"""
    return 2*x - 4

# 可視化
x = np.linspace(-1, 5, 100)
y = quadratic(x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 関数のプロット
axes[0].plot(x, y, 'b-', linewidth=2, label=r'$f(x) = (x-2)^2$')
axes[0].axvline(x=2, color='red', linestyle='--', label=r'最小点 $x^* = 2$')
axes[0].scatter([2], [0], color='red', s=100, zorder=5)
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('最適化問題: 関数の最小値を見つける')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 勾配のプロット
grad = quadratic_grad(x)
axes[1].plot(x, grad, 'g-', linewidth=2, label=r"$f'(x) = 2x - 4$")
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].axvline(x=2, color='red', linestyle='--', label=r"$f'(x^*) = 0$")
axes[1].fill_between(x[x < 2], 0, grad[x < 2], alpha=0.3, color='blue', label='勾配 < 0: 右へ移動')
axes[1].fill_between(x[x > 2], 0, grad[x > 2], alpha=0.3, color='orange', label='勾配 > 0: 左へ移動')
axes[1].set_xlabel('x')
axes[1].set_ylabel("f'(x)")
axes[1].set_title('勾配: 最小点では勾配がゼロ')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【ポイント】")
print("- 勾配が負 → パラメータを増やす方向が良い")
print("- 勾配が正 → パラメータを減らす方向が良い")
print("- 勾配がゼロ → 極値（最小点または最大点）")

---

## 2. 勾配降下法の原理

### 2.1 アルゴリズム

勾配降下法（Gradient Descent）は、勾配の **逆方向** にパラメータを更新することで関数を最小化します。

$$
\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)
$$

- $\eta$ (イータ): 学習率（Learning Rate）
- $\nabla L$: 損失関数の勾配

### 2.2 直感的な理解

山を下りる際、最も急な斜面を下っていくイメージです。

In [ ]:
def gradient_descent_1d(f, grad_f, x0, lr=0.1, n_steps=20):
    """
    1次元の勾配降下法
    
    Args:
        f: 目的関数
        grad_f: 勾配関数
        x0: 初期値
        lr: 学習率
        n_steps: ステップ数
    
    Returns:
        履歴（位置、関数値、勾配）
    """
    history = {'x': [x0], 'f': [f(x0)], 'grad': [grad_f(x0)]}
    x = x0
    
    for _ in range(n_steps):
        grad = grad_f(x)
        x = x - lr * grad  # 更新則
        history['x'].append(x)
        history['f'].append(f(x))
        history['grad'].append(grad_f(x))
    
    return history


# 勾配降下法の実行
history = gradient_descent_1d(quadratic, quadratic_grad, x0=0.0, lr=0.3, n_steps=15)

# 可視化
x = np.linspace(-1, 5, 100)
y = quadratic(x)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 最適化の軌跡
axes[0].plot(x, y, 'b-', linewidth=2, alpha=0.5)
axes[0].plot(history['x'], history['f'], 'ro-', markersize=8, linewidth=2, label='勾配降下の軌跡')
axes[0].scatter([history['x'][0]], [history['f'][0]], color='green', s=150, zorder=5, label=f'開始点 x={history["x"][0]:.1f}')
axes[0].scatter([history['x'][-1]], [history['f'][-1]], color='red', s=150, marker='*', zorder=5, label=f'終了点 x={history["x"][-1]:.3f}')
axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('勾配降下法の軌跡')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 収束の様子
steps = range(len(history['x']))
axes[1].plot(steps, history['x'], 'bo-', label='x')
axes[1].axhline(y=2, color='red', linestyle='--', label='最適値 x*=2')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('x')
axes[1].set_title('パラメータの収束')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【更新の履歴（最初の5ステップ）】")
print(f"{'Step':>5} {'x':>10} {'f(x)':>10} {'grad':>10}")
print("-" * 40)
for i in range(min(6, len(history['x']))):
    print(f"{i:>5} {history['x'][i]:>10.4f} {history['f'][i]:>10.4f} {history['grad'][i]:>10.4f}")

### 2.3 多次元への拡張

2次元以上の場合、勾配はベクトルになります。

$$
\nabla L(\theta) = \begin{pmatrix} \frac{\partial L}{\partial \theta_1} \\ \frac{\partial L}{\partial \theta_2} \\ \vdots \end{pmatrix}
$$

In [ ]:
def rosenbrock(x, y):
    """Rosenbrock関数: 最適化のベンチマーク関数"""
    return (1 - x)**2 + 100 * (y - x**2)**2

def rosenbrock_grad(x, y):
    """Rosenbrock関数の勾配"""
    dx = -2 * (1 - x) - 400 * x * (y - x**2)
    dy = 200 * (y - x**2)
    return np.array([dx, dy])


def gradient_descent_2d(f, grad_f, start, lr=0.001, n_steps=1000):
    """2次元の勾配降下法"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    
    for _ in range(n_steps):
        grad = grad_f(pos[0], pos[1])
        pos = pos - lr * grad
        path.append(pos.copy())
    
    return np.array(path)


# Rosenbrock関数での最適化
path = gradient_descent_2d(rosenbrock, rosenbrock_grad, start=(-1.5, 1.5), lr=0.001, n_steps=5000)

# 可視化
x = np.linspace(-2, 2, 200)
y = np.linspace(-1, 3, 200)
X, Y = np.meshgrid(x, y)
Z = rosenbrock(X, Y)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 等高線図
contour = axes[0].contour(X, Y, np.log(Z + 1), levels=30, cmap='viridis')
axes[0].plot(path[:, 0], path[:, 1], 'r.-', markersize=2, linewidth=1, alpha=0.7)
axes[0].scatter([path[0, 0]], [path[0, 1]], color='green', s=100, zorder=5, label='開始点')
axes[0].scatter([1], [1], color='red', s=100, marker='*', zorder=5, label='最適点 (1, 1)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Rosenbrock関数での勾配降下')
axes[0].legend()

# 損失の推移
losses = [rosenbrock(p[0], p[1]) for p in path]
axes[1].semilogy(losses, linewidth=2)
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('損失（対数スケール）')
axes[1].set_title('損失の推移')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"開始点: {path[0]}")
print(f"終了点: {path[-1]}")
print(f"最適点: (1, 1)")
print(f"最終損失: {losses[-1]:.6f}")

---

## 3. 学習率の影響

### 3.1 学習率とは

学習率 $\eta$ は、各ステップでどれだけ大きく移動するかを制御するハイパーパラメータです。

- **大きすぎる**: 発振、発散
- **小さすぎる**: 収束が遅い
- **適切**: 効率的に最適解に到達

In [ ]:
# 学習率の比較
learning_rates = [0.01, 0.1, 0.5, 1.1]
colors = ['blue', 'green', 'orange', 'red']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

x_range = np.linspace(-1, 5, 100)
y_range = quadratic(x_range)

for ax, lr, color in zip(axes, learning_rates, colors):
    history = gradient_descent_1d(quadratic, quadratic_grad, x0=0.0, lr=lr, n_steps=20)
    
    ax.plot(x_range, y_range, 'b-', linewidth=2, alpha=0.3)
    ax.plot(history['x'], history['f'], 'o-', color=color, markersize=8, linewidth=2)
    ax.axvline(x=2, color='red', linestyle='--', alpha=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.set_title(f'学習率 η = {lr}')
    ax.set_xlim(-2, 6)
    ax.set_ylim(-1, 20)
    ax.grid(True, alpha=0.3)
    
    # 最終位置を表示
    final_x = history['x'][-1]
    ax.annotate(f'x = {final_x:.2f}', (final_x, history['f'][-1]), 
                textcoords="offset points", xytext=(10, 10), fontsize=10)

plt.tight_layout()
plt.show()

print("【学習率の影響】")
print(f"η = 0.01: 収束が遅い（慎重すぎる）")
print(f"η = 0.1:  適切な収束")
print(f"η = 0.5:  速い収束")
print(f"η = 1.1:  発振（大きすぎる）")

In [ ]:
# 収束速度の詳細比較
learning_rates = [0.01, 0.1, 0.3, 0.5, 0.9]

fig, ax = plt.subplots(figsize=(10, 6))

for lr in learning_rates:
    history = gradient_descent_1d(quadratic, quadratic_grad, x0=0.0, lr=lr, n_steps=30)
    error = np.abs(np.array(history['x']) - 2)  # 最適値との差
    ax.semilogy(error, linewidth=2, label=f'η = {lr}')

ax.set_xlabel('ステップ')
ax.set_ylabel('|x - x*|（最適値との差）')
ax.set_title('学習率と収束速度の関係')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("- 学習率が大きいほど初期の収束は速い")
print("- ただし大きすぎると発振や発散のリスク")
print("- η = 1.0 を超えると不安定になる（この関数では）")

### 3.2 学習率と固有値の関係

二次関数の場合、収束条件は損失関数のヘッセ行列の固有値に依存します。

$$
0 < \eta < \frac{2}{\lambda_{\max}}
$$

- $\lambda_{\max}$: ヘッセ行列の最大固有値

In [ ]:
# 楕円形の二次関数での学習率の影響

def ellipse_loss(x, y, a=1, b=10):
    """楕円形の損失関数: L = a*x² + b*y²"""
    return a * x**2 + b * y**2

def ellipse_grad(x, y, a=1, b=10):
    """楕円形損失関数の勾配"""
    return np.array([2*a*x, 2*b*y])


# 異なる学習率での軌跡
starts = [(-3, 1)]
lrs = [0.01, 0.05, 0.09, 0.11]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# 等高線の準備
x = np.linspace(-4, 4, 100)
y = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x, y)
Z = ellipse_loss(X, Y)

for ax, lr in zip(axes, lrs):
    path = []
    pos = np.array(starts[0], dtype=float)
    path.append(pos.copy())
    
    for _ in range(50):
        grad = ellipse_grad(pos[0], pos[1])
        pos = pos - lr * grad
        path.append(pos.copy())
    
    path = np.array(path)
    
    ax.contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
    ax.plot(path[:, 0], path[:, 1], 'ro-', markersize=4, linewidth=1)
    ax.scatter([0], [0], color='red', s=100, marker='*', zorder=5)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    # 収束/発散の判定
    final_loss = ellipse_loss(path[-1, 0], path[-1, 1])
    status = "収束" if final_loss < 0.01 else ("発散" if final_loss > 100 else "振動")
    ax.set_title(f'η = {lr} ({status})')
    ax.set_xlim(-4, 4)
    ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("【楕円形損失関数の特性】")
print(f"ヘッセ行列: diag(2, 20)")
print(f"最大固有値 λ_max = 20")
print(f"理論上の安定条件: η < 2/20 = 0.1")

---

## 4. バッチ勾配降下法 vs 確率的勾配降下法

### 4.1 勾配降下法の種類

| 手法 | データ使用量 | 特徴 |
|------|------------|------|
| **Batch GD** | 全データ | 安定だが遅い |
| **SGD** | 1サンプル | 速いがノイジー |
| **Mini-batch GD** | 一部のデータ | バランスが良い |

In [ ]:
# 線形回帰での比較

# データ生成
np.random.seed(42)
n_samples = 100
X = np.random.randn(n_samples, 1)
true_w = 3.0
true_b = 1.0
y = true_w * X + true_b + np.random.randn(n_samples, 1) * 0.5


def compute_loss(w, b, X, y):
    """MSE損失"""
    pred = w * X + b
    return np.mean((pred - y) ** 2)


def compute_grad(w, b, X, y):
    """MSE損失の勾配"""
    pred = w * X + b
    error = pred - y
    grad_w = 2 * np.mean(error * X)
    grad_b = 2 * np.mean(error)
    return grad_w, grad_b


def train_batch_gd(X, y, lr=0.1, n_epochs=50):
    """バッチ勾配降下法"""
    w, b = 0.0, 0.0
    history = {'loss': [], 'w': [], 'b': []}
    
    for epoch in range(n_epochs):
        loss = compute_loss(w, b, X, y)
        grad_w, grad_b = compute_grad(w, b, X, y)
        
        w -= lr * grad_w
        b -= lr * grad_b
        
        history['loss'].append(loss)
        history['w'].append(w)
        history['b'].append(b)
    
    return w, b, history


def train_sgd(X, y, lr=0.01, n_epochs=50):
    """確率的勾配降下法"""
    w, b = 0.0, 0.0
    history = {'loss': [], 'w': [], 'b': []}
    n_samples = len(X)
    
    for epoch in range(n_epochs):
        # エポック開始時の損失を記録
        loss = compute_loss(w, b, X, y)
        history['loss'].append(loss)
        history['w'].append(w)
        history['b'].append(b)
        
        # ランダムな順序でサンプルを処理
        indices = np.random.permutation(n_samples)
        for i in indices:
            xi, yi = X[i:i+1], y[i:i+1]
            grad_w, grad_b = compute_grad(w, b, xi, yi)
            w -= lr * grad_w
            b -= lr * grad_b
    
    return w, b, history


def train_minibatch_gd(X, y, batch_size=16, lr=0.1, n_epochs=50):
    """ミニバッチ勾配降下法"""
    w, b = 0.0, 0.0
    history = {'loss': [], 'w': [], 'b': []}
    n_samples = len(X)
    
    for epoch in range(n_epochs):
        loss = compute_loss(w, b, X, y)
        history['loss'].append(loss)
        history['w'].append(w)
        history['b'].append(b)
        
        indices = np.random.permutation(n_samples)
        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            batch_idx = indices[start:end]
            xi, yi = X[batch_idx], y[batch_idx]
            grad_w, grad_b = compute_grad(w, b, xi, yi)
            w -= lr * grad_w
            b -= lr * grad_b
    
    return w, b, history


# 学習
w_batch, b_batch, hist_batch = train_batch_gd(X, y, lr=0.3, n_epochs=50)
w_sgd, b_sgd, hist_sgd = train_sgd(X, y, lr=0.01, n_epochs=50)
w_mini, b_mini, hist_mini = train_minibatch_gd(X, y, batch_size=16, lr=0.1, n_epochs=50)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 損失の推移
axes[0].plot(hist_batch['loss'], 'b-', linewidth=2, label='Batch GD')
axes[0].plot(hist_sgd['loss'], 'g-', linewidth=2, alpha=0.7, label='SGD')
axes[0].plot(hist_mini['loss'], 'r-', linewidth=2, label='Mini-batch GD')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('損失の推移')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# パラメータ空間での軌跡
axes[1].plot(hist_batch['w'], hist_batch['b'], 'b.-', linewidth=2, markersize=4, label='Batch GD')
axes[1].plot(hist_sgd['w'], hist_sgd['b'], 'g.-', linewidth=1, markersize=2, alpha=0.5, label='SGD')
axes[1].plot(hist_mini['w'], hist_mini['b'], 'r.-', linewidth=2, markersize=4, label='Mini-batch GD')
axes[1].scatter([true_w], [true_b], color='black', s=200, marker='*', zorder=5, label=f'真の値 ({true_w}, {true_b})')
axes[1].set_xlabel('w')
axes[1].set_ylabel('b')
axes[1].set_title('パラメータ空間での軌跡')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【結果】")
print(f"真の値:      w = {true_w:.2f}, b = {true_b:.2f}")
print(f"Batch GD:    w = {w_batch:.2f}, b = {b_batch:.2f}")
print(f"SGD:         w = {w_sgd:.2f}, b = {b_sgd:.2f}")
print(f"Mini-batch:  w = {w_mini:.2f}, b = {b_mini:.2f}")

### 4.2 バッチサイズの影響

In [ ]:
# バッチサイズの比較
batch_sizes = [1, 8, 32, 100]  # 100 = full batch

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for bs in batch_sizes:
    _, _, history = train_minibatch_gd(X, y, batch_size=bs, lr=0.1 if bs > 1 else 0.01, n_epochs=50)
    label = f'batch_size={bs}' if bs < 100 else 'Full batch'
    axes[0].plot(history['loss'], linewidth=2, label=label)

axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('バッチサイズと収束')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 勾配のノイズ（分散）
n_simulations = 100
grad_vars = []

for bs in batch_sizes:
    grads = []
    for _ in range(n_simulations):
        idx = np.random.choice(n_samples, bs, replace=False)
        grad_w, _ = compute_grad(2.0, 0.5, X[idx], y[idx])  # 固定点での勾配
        grads.append(grad_w)
    grad_vars.append(np.var(grads))

axes[1].bar(range(len(batch_sizes)), grad_vars, tick_label=[str(bs) for bs in batch_sizes])
axes[1].set_xlabel('バッチサイズ')
axes[1].set_ylabel('勾配の分散')
axes[1].set_title('バッチサイズと勾配のノイズ')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("- バッチサイズが小さい → 勾配のノイズが大きい")
print("- ノイズは正則化効果があり、汎化性能向上に寄与することも")
print("- 大きなバッチは安定だがメモリ消費が多い")

---

## 5. 損失曲面の可視化

### 5.1 3D損失曲面

In [ ]:
# パラメータ空間での損失曲面
w_range = np.linspace(-1, 5, 100)
b_range = np.linspace(-2, 4, 100)
W, B = np.meshgrid(w_range, b_range)
Z = np.zeros_like(W)

for i in range(W.shape[0]):
    for j in range(W.shape[1]):
        Z[i, j] = compute_loss(W[i, j], B[i, j], X, y)

fig = plt.figure(figsize=(14, 5))

# 3Dプロット
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax1.plot_surface(W, B, Z, cmap='viridis', alpha=0.8, edgecolor='none')
ax1.set_xlabel('w')
ax1.set_ylabel('b')
ax1.set_zlabel('Loss')
ax1.set_title('損失曲面 (3D)')
ax1.view_init(elev=30, azim=45)

# 等高線プロット
ax2 = fig.add_subplot(1, 2, 2)
contour = ax2.contour(W, B, Z, levels=30, cmap='viridis')
ax2.scatter([true_w], [true_b], color='red', s=100, marker='*', zorder=5, label='真の値')
ax2.set_xlabel('w')
ax2.set_ylabel('b')
ax2.set_title('損失曲面 (等高線)')
ax2.legend()
plt.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

---

## 6. 凸関数と非凸関数

### 6.1 凸関数の定義

関数 $f$ が **凸** であるとは、任意の2点 $x, y$ と $0 \le \lambda \le 1$ に対して：

$$
f(\lambda x + (1-\lambda) y) \le \lambda f(x) + (1-\lambda) f(y)
$$

幾何学的には、グラフ上の任意の2点を結ぶ線分がグラフより上にあることを意味します。

In [ ]:
# 凸関数 vs 非凸関数

def convex_func(x):
    """凸関数: f(x) = x²"""
    return x**2

def nonconvex_func(x):
    """非凸関数: f(x) = x⁴ - 2x² + 0.5"""
    return x**4 - 2*x**2 + 0.5

x = np.linspace(-2, 2, 200)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 凸関数
y_convex = convex_func(x)
axes[0].plot(x, y_convex, 'b-', linewidth=2)

# 凸性の可視化: 2点を結ぶ線分
x1, x2 = -1.5, 1.0
y1, y2 = convex_func(x1), convex_func(x2)
axes[0].plot([x1, x2], [y1, y2], 'r--', linewidth=2, label='2点を結ぶ線分')
axes[0].scatter([x1, x2], [y1, y2], color='red', s=100, zorder=5)
axes[0].fill_between(np.linspace(x1, x2, 100), 
                     convex_func(np.linspace(x1, x2, 100)),
                     np.linspace(y1, y2, 100),
                     alpha=0.3, color='green', label='線分より下')

axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('凸関数: 局所最小 = 大域最小')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 非凸関数
y_nonconvex = nonconvex_func(x)
axes[1].plot(x, y_nonconvex, 'b-', linewidth=2)

# 極値点をマーク
x_min_local = np.array([-1, 1])
y_min_local = nonconvex_func(x_min_local)
axes[1].scatter(x_min_local, y_min_local, color='green', s=100, zorder=5, label='局所最小点')

x_max_local = np.array([0])
y_max_local = nonconvex_func(x_max_local)
axes[1].scatter(x_max_local, y_max_local, color='red', s=100, zorder=5, label='局所最大点')

axes[1].set_xlabel('x')
axes[1].set_ylabel('f(x)')
axes[1].set_title('非凸関数: 複数の極値')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【凸関数の利点】")
print("- 局所最小点 = 大域最小点（唯一解）")
print("- 勾配降下法で必ず最適解に到達")
print("")
print("【非凸関数の課題】")
print("- 複数の局所最小点が存在")
print("- 初期値によって到達する解が異なる")
print("- ニューラルネットワークの損失関数は一般に非凸")

In [ ]:
# 非凸関数での勾配降下法の挙動

def nonconvex_grad(x):
    return 4*x**3 - 4*x

# 異なる初期値からの最適化
initial_points = [-2.0, -0.5, 0.5, 2.0]
colors = ['red', 'green', 'blue', 'orange']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

x = np.linspace(-2.5, 2.5, 200)
y = nonconvex_func(x)

for ax in axes:
    ax.plot(x, y, 'k-', linewidth=2, alpha=0.5)

for x0, color in zip(initial_points, colors):
    history = gradient_descent_1d(nonconvex_func, nonconvex_grad, x0=x0, lr=0.1, n_steps=30)
    
    axes[0].plot(history['x'], history['f'], 'o-', color=color, markersize=6, 
                 linewidth=2, label=f'x₀ = {x0}')
    
    axes[1].plot(history['x'], 'o-', color=color, markersize=6, 
                 linewidth=2, label=f'x₀ = {x0}')

axes[0].set_xlabel('x')
axes[0].set_ylabel('f(x)')
axes[0].set_title('非凸関数での最適化軌跡')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].axhline(y=-1, color='gray', linestyle='--', alpha=0.5, label='局所最小 x=-1')
axes[1].axhline(y=1, color='gray', linestyle='--', alpha=0.5, label='局所最小 x=1')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('x')
axes[1].set_title('パラメータの収束')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print("初期値によって収束先が異なる：")
for x0, color in zip(initial_points, colors):
    history = gradient_descent_1d(nonconvex_func, nonconvex_grad, x0=x0, lr=0.1, n_steps=30)
    print(f"  x₀ = {x0:5.1f} → x* = {history['x'][-1]:6.3f}")

---

## 7. 収束条件と停止基準

### 7.1 一般的な停止基準

1. **反復回数**: 最大エポック数に達したら停止
2. **勾配ノルム**: $\|\nabla L\| < \epsilon$
3. **パラメータ変化**: $\|\theta_{t+1} - \theta_t\| < \epsilon$
4. **損失変化**: $|L_{t+1} - L_t| < \epsilon$

In [ ]:
def gradient_descent_with_stopping(f, grad_f, x0, lr=0.1, max_steps=1000, 
                                   grad_tol=1e-6, param_tol=1e-8, loss_tol=1e-10):
    """
    停止条件付き勾配降下法
    
    Returns:
        final_x, history, stop_reason
    """
    history = {'x': [x0], 'f': [f(x0)], 'grad_norm': [abs(grad_f(x0))]}
    x = x0
    stop_reason = "max_steps"
    
    for step in range(max_steps):
        grad = grad_f(x)
        grad_norm = abs(grad)
        
        # 勾配ノルムによる停止
        if grad_norm < grad_tol:
            stop_reason = f"勾配ノルム < {grad_tol}"
            break
        
        # パラメータ更新
        x_new = x - lr * grad
        
        # パラメータ変化による停止
        if abs(x_new - x) < param_tol:
            stop_reason = f"パラメータ変化 < {param_tol}"
            break
        
        # 損失変化による停止
        loss_old = f(x)
        loss_new = f(x_new)
        if abs(loss_new - loss_old) < loss_tol:
            stop_reason = f"損失変化 < {loss_tol}"
            break
        
        x = x_new
        history['x'].append(x)
        history['f'].append(f(x))
        history['grad_norm'].append(grad_norm)
    
    return x, history, stop_reason


# 実行
x_final, history, reason = gradient_descent_with_stopping(
    quadratic, quadratic_grad, x0=0.0, lr=0.3, 
    max_steps=1000, grad_tol=1e-6
)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

steps = range(len(history['x']))

axes[0].plot(steps, history['x'], 'b.-')
axes[0].axhline(y=2, color='red', linestyle='--')
axes[0].set_xlabel('ステップ')
axes[0].set_ylabel('x')
axes[0].set_title('パラメータの収束')
axes[0].grid(True, alpha=0.3)

axes[1].semilogy(steps, history['f'], 'b.-')
axes[1].set_xlabel('ステップ')
axes[1].set_ylabel('f(x)')
axes[1].set_title('損失')
axes[1].grid(True, alpha=0.3)

axes[2].semilogy(steps, history['grad_norm'], 'g.-')
axes[2].axhline(y=1e-6, color='red', linestyle='--', label='停止閾値')
axes[2].set_xlabel('ステップ')
axes[2].set_ylabel('|勾配|')
axes[2].set_title('勾配ノルム')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"【結果】")
print(f"最終値: x = {x_final:.8f}")
print(f"ステップ数: {len(history['x']) - 1}")
print(f"停止理由: {reason}")

---

## 8. 演習問題

### 演習 8.1: Booth関数の最適化

Booth関数 $f(x, y) = (x + 2y - 7)^2 + (2x + y - 5)^2$ を勾配降下法で最適化してください。
最適点は $(1, 3)$ です。

In [ ]:
# 演習 8.1: 解答欄

def booth(x, y):
    # TODO: 実装
    pass

def booth_grad(x, y):
    # TODO: 実装
    pass

# TODO: 勾配降下法を実行し、軌跡を可視化
pass

### 演習 8.2: 学習率のチューニング

Rosenbrock関数に対して、様々な学習率を試し、最も効率的に収束する学習率を見つけてください。

In [ ]:
# 演習 8.2: 解答欄

# TODO: 学習率 0.0001, 0.0005, 0.001, 0.005 を比較

pass

### 演習 8.3: バッチサイズとノイズ

異なるバッチサイズでの勾配分散を計算し、バッチサイズが大きくなるにつれて分散がどのように減少するかを確認してください。理論的には分散は $O(1/n)$ で減少します。

In [ ]:
# 演習 8.3: 解答欄

# TODO: バッチサイズ 1, 2, 4, 8, 16, 32, 64 での勾配分散を計算

pass

---

## 9. まとめと次のステップ

### このノートブックで学んだこと

1. **最適化問題の定式化**: $\theta^* = \arg\min L(\theta)$

2. **勾配降下法**: $\theta_{t+1} = \theta_t - \eta \nabla L$

3. **学習率の重要性**: 大きすぎると発散、小さすぎると収束が遅い

4. **バッチサイズのトレードオフ**:
   - 大きい → 安定だが遅い
   - 小さい → 速いがノイジー

5. **凸 vs 非凸**: ニューラルネットは非凸で複数の局所解が存在

### 次のノートブック（111: Momentum と Nesterov）への橋渡し

基本的なSGDには以下の問題があります：

- **振動**: 急峻な谷では行ったり来たりする
- **鞍点での停滞**: 勾配が小さい領域で動きが遅くなる

次のノートブックでは、これらの問題を解決する **Momentum** と **Nesterov加速** を学びます。

---

## 参考文献

1. Bottou, L. (2010). Large-scale machine learning with stochastic gradient descent. *COMPSTAT*.
2. Ruder, S. (2016). An overview of gradient descent optimization algorithms. *arXiv:1609.04747*.
3. Boyd, S., & Vandenberghe, L. (2004). *Convex Optimization*. Cambridge University Press.